<a href="https://colab.research.google.com/github/gkh2004/Mini-project-12/blob/main/audio_to_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install SpeechRecognition pydub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 20.7 MB/s eta 0:00:00


In [2]:
import speech_recognition as sr
import os
import subprocess
from google.colab import files
from pydub import AudioSegment
from pydub.silence import split_on_silence

In [3]:
files.upload()

Saving audio.unknown to audio.unknown


{'audio.unknown': b'\x00\x00\x00\x18ftyp3gp4\x00\x00\x00\x00isom3gp4\x00\x00\x00\x01mdat\x00\x00\x00\x00\x00\x00\x89\xef\x01@B\x80\xa3\x7f\xf8\x85---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------.\xf4\xf1\nZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZ^\x01 \x9c\xda\xb3dW\xab13z\xa4&&oFT\xe2\xe3UW$g\xf1\xc6\xb8j\xe5\xa5\xd7\x7f\xdb\xae=\xbcq\xc4K\x99\xcf\xf6\xeb^}y\xd5\x97+\xb6\x8a\x05\xcf\x00\xfc\x8ei\xdf\xec\x98N\xb9\x0f\xd83\x8b\xc6\xbf\t\xa3\xdb4I?L\x82\x15%\x9cr\x0f)\x88\xa4\xb0\x07\xd0\x823<\xc1`z\'\x00\xd9\xe0\xc2\x0b\x7f\x8a\xb61\xdfv\xf1\xf5p\x0e\xb8y\x99B?T\xaa\xf0X\'\xaf\xb4FZ\x0c\x89\xdf\x8co\xbd\t!\xd1\xf7\xd6H\xce~\xcfL\xbf\x1d\xaf\xac\x8b\n\x95A\x15m6)\x1e1nm@0\xb2\xd1e\x0bk\xb6\xec\x8e\x8bJ\xd29\x05\x82\xe7\x9c\

In [4]:
# convert mp3 to wav file
subprocess.call(['ffmpeg', '-i', '01.mp3','wav_file.wav'])

1

In [5]:
# initialize the recognizer
r = sr.Recognizer()

# a function that splits the audio file into chunks
# and applies speech recognition

def get_large_audio_transcription(path):
    """
    Splitting the large audio file into chunks
    and apply speech recognition on each of these chunks
    """
    # open the audio file using pydub
    sound = AudioSegment.from_wav(path)
    # split audio sound where silence is 700 miliseconds or more and get chunks
    chunks = split_on_silence(sound,
        # experiment with this value for your target audio file
        min_silence_len = 500,
        # adjust this per requirement
        silence_thresh = sound.dBFS-14,
        # keep the silence for 1 second, adjustable as well
        keep_silence=500,
    )
    folder_name = "audio-chunks"
    # create a directory to store the audio chunks
    if not os.path.isdir(folder_name):
        os.mkdir(folder_name)
    whole_text = ""
    # process each chunk
    for i, audio_chunk in enumerate(chunks, start=1):
        # export audio chunk and save it in
        # the `folder_name` directory.
        chunk_filename = os.path.join(folder_name, f"chunk{i}.wav")
        audio_chunk.export(chunk_filename, format="wav")
        # recognize the chunk
        with sr.AudioFile(chunk_filename) as source:
            audio_listened = r.record(source)
            # try converting it to text
            try:
                text = r.recognize_google(audio_listened)
            except sr.UnknownValueError as e:
                print("Error:", str(e))
            else:
                text = f"{text.capitalize()}. "
                print(chunk_filename, ":", text)
                whole_text += text
    # return the text for all chunks detected
    return whole_text

In [ ]:
path = "/content/wav_file.wav"
print("\nFull text:", get_large_audio_transcription(path))